In [6]:
# res_path = "../e2efold_data/results/"


In [ ]:
import bio

In [11]:
import numpy as np

# Assuming your .npy file is named 'example.npy'
file_path = "/home/eric/RNA-FM/recover_experiments/e2efold_data/results/pred_ct/5s_Acanthamoeba-castellanii-1.ct"
# Load the data from the .npy file
loaded_data = np.load(file_path)
np.size(loaded_data, 0), np.size(loaded_data, 1)
loaded_data[:10][:10]
# Now 'loaded_data' contains the contents of the .npy file as a NumPy array
# print(loaded_data)

ValueError: Cannot load file containing pickled data when allow_pickle=False

In [21]:
import collections
import _pickle as cPickle

RNA_SS_data = collections.namedtuple('RNA_SS_data', 
    'seq ss_label length name pairs')

file_path = "/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/all_1800.pickle"

with open(file_path, 'rb') as file:
    loaded_data = cPickle.load(file)

    

In [20]:
len(loaded_data)

55

In [17]:
import collections
import _pickle as cPickle

RNA_SS_data = collections.namedtuple('RNA_SS_data', 
    'seq ss_label length name pairs')

file_path = "/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/all_1800.pickle"

with open(file_path, 'rb') as file:
    loaded_data = cPickle.load(file)

In [18]:
seq_path = "/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/archiveII.seq"
with open(seq_path, 'rb') as file:
    for line in file:
        # Assuming each line contains a single RNA sequence
        if len(line) == 0:
            print(11111111111111)

In [27]:
def convert_seq_to_fasta(seq_file_path, fasta_file_path):
    line_name = None
    line_seq = None
    with open(seq_file_path, 'r') as seq_file, open(fasta_file_path, 'w') as fasta_file:
        for i, line in enumerate(seq_file, start=1):
            # Assuming each line contains a single RNA sequence
            if line_name == None:
                line = line.replace(".ct", "")
                line_name = line
                continue
            if line_seq == None:
                line_seq = line
                fasta_file.write(f">{line_name}{line_seq}")
                
                line_name, line_seq = None, None
            # Write the header and sequence to the FASTA file
            # fasta_file.write(f"{header}\n{sequence}\n")

# Example usage:
seq_file_path = '/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/archiveII.seq'
fasta_file_path = '/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/archiveII.fasta'
convert_seq_to_fasta(seq_file_path, fasta_file_path)

In [89]:
# def compute_precision_at_l5(self, sequence_lengths, prediction, labels):
#     with torch.no_grad():
#         # valid_mask = labels != self._ignore_index
#         # seqpos = torch.arange(valid_mask.size(1), device=sequence_lengths.device)
#         # x_ind, y_ind = torch.meshgrid(seqpos, seqpos)
#         # valid_mask &= ((y_ind - x_ind) >= 6).unsqueeze(0)
#         # probs = F.softmax(prediction, 3)[:, :, :, 1]
#         # valid_mask = valid_mask.type_as(probs)
#         correct = 0
#         total = 0
#         for length, prob, label, mask in zip(sequence_lengths, probs, labels, valid_mask):
#             masked_prob = (prob * mask).view(-1)
#             most_likely = masked_prob.topk(length // 5, sorted=False)
#             selected = label.view(-1).gather(0, most_likely.indices)
#             correct += selected.sum().float()
#             total += selected.numel()
#         return correct / total

import torch
import os
import numpy as np
def get_prob_from_results(folder_path: str):

    res_dict = dict()

    for filename in os.listdir(folder_path):
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Check if it is a file (not a subfolder)
        if os.path.isfile(file_path):
            rna_name = filename.replace(".npy", "")
            data = np.load(file_path)
            data = torch.from_numpy(data)
            res_dict[rna_name] = data
    return res_dict



import collections
import _pickle as cPickle

def get_pair_from_npy(file_paths: list):

    res_dict = dict()

    for file_path in file_paths:
        with open(file_path, 'rb') as file:
            loaded_data = cPickle.load(file)
            for data in loaded_data:
                res_dict[data.name.replace(".ct", "")] = data
    return res_dict

In [117]:
probs = get_prob_from_results("/home/eric/RNA-FM/recover_experiments/e2efold_data/results/r-ss")

In [118]:
labels = get_pair_from_npy(["/home/eric/RNA-FM/recover_experiments/e2efold_data/archiveII_all/all_600.pickle"])

In [123]:
len(labels)

3911

In [119]:
import torch

def get_pair_using_top5(prob, label):
    prob = prob.cuda()
    seqpos = torch.arange(label.length, device='cuda')
    x_ind, y_ind = torch.meshgrid(seqpos, seqpos)
    valid_mask = ((y_ind - x_ind) >= 6).cuda()
    masked_prob = (prob * valid_mask).view(-1)
    most_likely = masked_prob.topk(label.length // 5, sorted=False)

    pairs = label.pairs
    label = torch.zeros((label.length, label.length)).cuda()

    for pair in pairs:
        label[pair[0], pair[1]] = 1
        label[pair[1], pair[0]] = 1
    selected = label.view(-1).gather(0, most_likely.indices)
    return selected


In [120]:
res = []
for name in list(labels.keys()):
    if name not in probs:
        continue
    prob = probs[name]
    label = labels[name]
    selected = get_pair_using_top5(prob, label).tolist()
    # correct += selected.sum().float()
    # total += selected.numel()
    # print(selected)

    res = res + selected


In [127]:
from sklearn.metrics import f1_score, average_precision_score, recall_score
f1_score(res, [1 for _ in range(len(res))] ), average_precision_score(res, [1 for _ in range(len(res))]), recall_score(res, [1 for _ in range(len(res))], average='macro')

(0.8440694430325416, 0.730207742969376)

In [122]:
correct/total

tensor(0.7138, device='cuda:0')

In [ ]:
import numpy as np